In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataclasses import dataclass, field

from typing import Dict, List, Tuple, Optional, Callable, Any

import math
import numpy as np
from PIL import Image

from pysolotools.consumers import Solo
from pysolotools.converters.solo2coco import SOLO2COCOConverter
from pysolotools.core.models import KeypointAnnotationDefinition, RGBCameraCapture
from pysolotools.core.models import BoundingBox2DLabel, BoundingBox2DAnnotation
from pysolotools.core.models import BoundingBox3DLabel, BoundingBox3DAnnotation
from pysolotools.core.models import Frame, Capture
from scipy.spatial.transform import Rotation as R

from torchvision.datasets import ImageFolder
from torchvision.models import swin_v2_t, Swin_V2_T_Weights
from torchvision.models import swin_v2_b, Swin_V2_B_Weights
# from torch.utils.data import ConcatDataset, DataLoader
from collections import OrderedDict
from torch.utils.data import DataLoader

import torch
from torch import nn, Tensor

from torch.nn import functional as F

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import LearningRateMonitor

import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from scipy.io import savemat

import reader
import network
from my_trainer import SetCriterion
torch.set_float32_matmul_precision('medium')
# %matplotlib ipympl

In [3]:
train_folder = 'D:/Unity/dataset/solo'
# training_dir = './data/train'
# testing_dir = './data/test'

In [4]:
train_loader = reader.UnityDataset.from_unity_to_loader(root=train_folder, batch_size=8)

for batch in train_loader:
    image_dicts, object_list = batch

    # for key, image in image_dicts.items():
    #     print(key, image.shape)
    # print('-'*30)
    # for targets in object_list:
    #     print(targets.position)

    break

In [ ]:
net = network.CVMPCA(
    captures=train_loader.dataset.captures,
    capture_lookup=train_loader.dataset.capture_lookup,
    num_classes=len(train_loader.dataset.category_lookup), 
    num_layers=4, num_query=200,
    pc_range=[[-11, 7], [0, 3], [-7, 7]]
)
cat_list, position_list, los_list = net.forward(image_dicts)

# num_layers, num_query = cat_list.shape[1], cat_list.shape[2]
# index_tuple_list = []
# with torch.no_grad():

#     # for each sample in the batch
#     for label, (cat, pos, los) in zip(object_list, zip(cat_list, position_list, los_list)):
#         cat, pos, los = cat[0], pos[0], los[0]  # use the same indices for all levels
#         C = np.zeros((label.category.shape[0], cat.shape[0]))  # (K, K_hat)

#         for i in range(label.category.shape[0]):
#             # print(cat.shape, label.category[i].shape)
#             cat_delta = F.binary_cross_entropy_with_logits(
#                 cat, label.category[i].unsqueeze(0).expand(num_query, -1), reduction='none'
#             ).mean(-1)
#             pos_delta = F.mse_loss(
#                 pos*(net.loss.pc_range[:, 1] - net.loss.pc_range[:, 0]) + net.loss.pc_range[:, 0],
#                 label.position[i].unsqueeze(0).expand(num_query, -1), reduction='none'
#             ).mean(-1)
#             los_delta = F.binary_cross_entropy_with_logits(
#                 los, label.los[i].unsqueeze(0).expand(num_query, -1), reduction='none'
#             ).mean(-1)
#             # print(cat_delta.shape, pos_delta.shape, los_delta.shape)

#             C[i] = cat_delta + pos_delta + los_delta

#         label_indices, cat_indices = linear_sum_assignment(C)
#         index_tuple_list.append((label_indices, cat_indices))


# cls_losses, reg_losses, los_losses = [], [], []
# for i, indices in enumerate(index_tuple_list):
#     label_indices, cat_indices = indices
#     cls_loss = F.binary_cross_entropy_with_logits(
#         cat_list[i, :, cat_indices],
#         object_list[i].category[label_indices].unsqueeze(0).expand(6, -1, -1)
#     )
#     reg_loss = F.mse_loss(
#         position_list[i, :, cat_indices],#*(net.loss.pc_range[:, 1] - net.loss.pc_range[:, 0]) + net.loss.pc_range[:, 0],
#         object_list[i].position[label_indices].unsqueeze(0).expand(6, -1, -1)
#     )
#     los_loss = F.binary_cross_entropy_with_logits(
#         los_list[i, :, cat_indices],
#         object_list[i].los[label_indices].unsqueeze(0).expand(6, -1, -1)
#     )

for batch in train_loader:
    image_dicts, object_list = batch
    # net(image_dicts)
    loss = net.training_step(batch, 1)
    # print(loss)
    break


In [13]:
logger = TensorBoardLogger(
    "D:/runs", name="CVMPCA"
)
trainer = pl.Trainer(
    max_epochs=50, precision='16-mixed',
    gradient_clip_val=35,
    logger=logger,
    # callbacks=[
    #     LearningRateMonitor(logging_interval='step')
    # ],
    log_every_n_steps=20,
    # profiler="simple",
)
net = network.CVMPCA(
    captures=train_loader.dataset.captures,
    capture_lookup=train_loader.dataset.capture_lookup,
    num_classes=len(train_loader.dataset.category_lookup), 
    num_layers=6, num_query=200,
    pc_range=[[-110, 70], [0, 30], [-70, 70]]
)
# loss_func = SetCriterion(
#     num_ue=50, num_sbs=2, num_class=1,
#     num_layers=3, pc_range=[[-5, 5], [0, 3], [-5, 5]]
# )

trainer.fit(net, train_dataloaders=train_loader)


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type         | Params
---------------------------------------------
0 | cameras     | ModuleDict   | 0     
1 | single_view | SingleView   | 28.1 M
2 | query       | Embedding    | 19.2 K
3 | to_anchor   | Linear       | 291   
4 | decoder     | Decoder      | 567 K 
5 | loss        | SetCriterion | 0     
---------------------------------------------
28.6 M    Trainable params
0         Non-trainable params
28.6 M    Total params
114.554   Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
pc_range_tensor = torch.tensor([[-11, 7], [0, 3], [-7, 7]])

In [ ]:
pc_range_tensor[:, 1] - pc_range_tensor[:, 0]

In [ ]:

scores, positions, visibilities = net({key: image.cuda() for key, image in image_dicts.items()})
gt_scores, gt_positions, gt_visibilities = targets

In [ ]:
indinces = scores[:, -1, :, 0].argmax(dim=-1)
positions = positions.cpu()
# print(positions_)
print(positions.shape)
# gt_positions

pc_range = torch.tensor([[-5, 5], [0, 3], [-5, 5]], dtype=torch.float)
for i, idx in enumerate(indinces.cpu()):
    pos = positions[i, -1, idx]
    # print(positions.shape, pos.shape)
    pos = pos*(pc_range[:, 1] - pc_range[:, 0]) + pc_range[:, 0]
    print(pos.detach())
    print(gt_positions[i])
    print()

In [ ]:
pc_range

In [ ]:
scores.shape

In [ ]:
single_view = network.SingleView()

visual_features = {
    key: single_view(image)
    for key, image in image_dicts.items()
}
for features in visual_features.values():
    for feature in features.values():
        print(feature.shape)
    print('-'*30)

In [ ]:
head = network.MyTransformer(
    captures=train_loader.dataset.captures,
    num_classes=1, num_levels=4,
    num_layers=4, num_query=50,
    embed_dim=96, num_feature_levels=4,
)
loss_func = SetCriterion(
    num_ue=5, num_sbs=2, num_category=1,
)

scores, positions, visibilities = head(visual_features)

loss_func(
    (scores, positions, visibilities),
    
)

In [ ]:
trainer = pl.Trainer(max_epochs=200, precision=16)
net = network.CVMPCA(
    captures=train_loader.dataset.captures,
    space_size=[[-5, 5], [0, 3], [-5, 5]],
    voxel_size=[1., 1., 1.],
    num_neck_layers=1,
    num_ue=5, num_category=1,
)

trainer.fit(net, train_dataloaders=train_loader)

In [ ]:

def compute_grid(space_size, voxel_size, device=None):
    nBins = [math.floor((space[1] - space[0])//voxel)+1 for space, voxel in zip(space_size, voxel_size)]
    
    X = torch.linspace(space_size[0][0], space_size[0][1], nBins[0], device=device)
    Y = torch.linspace(space_size[1][0], space_size[1][1], nBins[1], device=device)
    Z = torch.linspace(space_size[2][0], space_size[2][1], nBins[2], device=device)
    
    gridx, gridy, gridz = torch.meshgrid(X, Y, Z)
    gridx = gridx.contiguous().view(-1, 1)
    gridy = gridy.contiguous().view(-1, 1)
    gridz = gridz.contiguous().view(-1, 1)
    grid = torch.cat([gridx, gridy, gridz], dim=1)
    return grid

grid = compute_grid(
    space_size=[[-4, 4], [0, 3], [-4, 4]],
    voxel_size=[1, 1, 1],
)
# grid

In [ ]:
loss_func = SetCriterion(
    num_ue=5, num_sbs=2, num_category=1,
)
model = network.CVMPCA(
    captures=train_loader.dataset.captures,
    space_size=[[-5, 5], [0, 3], [-5, 5]],
    voxel_size=[.2, .2, .2],
    num_neck_layers=1,
    num_ue=5, num_category=1,
)
for batch in train_loader:
    image_dict, gt_ues = batch

    pred = model(image_dict, 5)
    # loss = loss_func(pred, gt_ues)

    # print(loss)
    break

In [ ]:

with torch.no_grad():
    pred_ues, pred_indices = pred
    score_list, pos_list, vis_list = gt_ues

    B, num_ue = pred_ues.shape[:2]

    indices = []
    # for each sample in a batch
    for p_ue, p_indices, _, gt_p, gt_v in zip(pred_ues, pred_indices, score_list, pos_list, vis_list):
        p_anchor = p_indices
        
        scores = p_ue[:, 0].sigmoid()
        positions = p_ue[:, 1:4]
        visibilities = p_ue[:, 4:]
        
        cost_pos = torch.cdist(positions, gt_p, p=1)
        cost_vis = []
        for p_v in visibilities:
            cost_vis.append(torch.tensor([
                F.binary_cross_entropy_with_logits(p_v, t_v) for t_v in gt_v
            ], device=pred_ues[-1].device))
        cost_vis = torch.stack(cost_vis)

        C = scores.unsqueeze(-1) + cost_pos + cost_vis
        indices.append(linear_sum_assignment(C))


In [ ]:
model = network.CVMPCA(
    captures=train_loader.dataset.captures,
    space_size=[[-5, 5], [0, 3], [-5, 5]],
    voxel_size=[.75, .75, .75],
    num_neck_layers=1,
    num_category=1,
    num_ue=5, 
)
loss = SetCriterion(
    num_ue=5, num_sbs=2, num_category=1,
)
print('grid', model.grid.shape)
for batch in train_loader:
    image_dict, _ = batch
    hierachical_visual_features = {key: model.backbone(image) for key, image in image_dict.items()}
    
    last_feature = {key: features[-1] for key, features in hierachical_visual_features.items()}
    xs = {}
    for key, feature in last_feature.items():
        B, H, W, D = feature.size()
        # convert the 3D positions into pixel coordinate
        coor = model.cameras[key](model.grid, [H, W, 1])
        # record out-of-bound 3D positions
        bounding_x = coor[:, 0] > H
        bounding_y = coor[:, 1] > W
        bounding = torch.logical_and(bounding_x, bounding_y)
        # make sure the pixel coordinate stay within the image for the 'gather' api
        coor[:, 0] = torch.clamp(coor[:, 0], 0, H - 1)
        coor[:, 1] = torch.clamp(coor[:, 1], 0, W - 1)

        N = coor.size(0)
        flat_indices = coor[:, 0] * H + coor[:, 1]
        selected_features = torch.gather(
            feature.view(B, H * W, D), dim=1,
            index=flat_indices.long().view(1, N, 1).expand(B, -1, D)
        )
        # replace out-of-bound position with zero (by multiplying with zero)
        selected_features = selected_features * bounding.logical_not().unsqueeze(0).unsqueeze(-1)
        xs[key] = selected_features

    voxels = torch.stack([x for x in xs.values()], dim=1).mean(1)
    print(voxels.shape)
    proposal = model.proposal(voxels)
    print(proposal.shape)

    k = 5
    top_proposal, top_indices = torch.topk(proposal[:, :, 0], k, dim=1, largest=True, sorted=True)
    indices_x = top_indices // (model.nBins[0] * model.nBins[1])
    indices_y = (top_indices % (model.nBins[0] * model.nBins[1])) // model.nBins[0]
    indices_z = (top_indices % (model.nBins[0] * model.nBins[1])) % model.nBins[0]
    indices = torch.stack([indices_x, indices_y, indices_z], dim=-1)
    print(indices.shape)
    break

In [ ]:
trainer = pl.Trainer(max_epochs=200, precision=16)
net = network.CVMPCA(
    captures=train_loader.dataset.captures,
    space_size=[[-5, 5], [0, 3], [-5, 5]],
    voxel_size=[1., 1., 1.],
    num_neck_layers=1,
    num_ue=5, num_category=1,
)

trainer.fit(net, train_dataloaders=train_loader)